In [1]:
require('nn')
require('hdf5')
require('optim')

In [2]:
f = hdf5.open("data.hdf5", "r")
X_train = f:read("train_input"):all()
Y_train = f:read("train_output"):all()
X_valid = f:read("valid_input"):all()
q_valid = f:read("valid_q"):all()
X_test = f:read("test_input"):all()
q_test = f:read("test_q"):all()
kaggle_valid = f:read("valid_kaggle"):all()
nclasses = f:read('nclasses'):all():long()[1]
nfeatures = f:read('nfeatures'):all():long()[1]
f:close()

window_size = X_train:size(2)
n_actual = nfeatures

In [25]:
--changes index to account for window size
function changeIndex(X, window_size, nfeatures)
    for i=1,X:size(2) do
        X:select(2,i):add((i-1)*nfeatures)
    end
    return X
end

function lookup(X, Y, win_size, C, K)
    W = torch.zeros(C, win_size*(K))
    for i=1,X:size(1) do
        for j=1,X:size(2) do
            local row = Y[i]
            local col = X[i][j]
            W[row][col] = W[row][col] + 1
        end
    end
    return W
end

function CBM(fc, Y, alpha, V)
    --calculate log posterior
    local cc = torch.zeros(1, V)
    local clp = torch.zeros(1, V)

    --collapse column wise so Nx1
    local hist = torch.histc(Y:double(), 10001)
    cc:add(torch.reshape(hist, 1, 10001))
    --THIS IS ALPHA
    --smoothing
    --cc:add(alpha)
    fc:add(alpha)
    local scc = fc:sum(2)
    
    fc:log()
    scc:log()
    
    scc:expand(scc, scc:size(1), fc:size(2))
    
    --calculate log prior
    --collapse down so 1x1
    local total = cc:sum(2)
    cc:div(total[1][1])
    cc:log()
    --clp:csub(math.log(Y:size(2)))
    
    return fc:csub(scc), cc
end

function predict(X, W, b)
  local jll = torch.zeros(X:size(1), W:size(1))
  for i = 1, X:size(1) do
    jll[i] = W:index(2, X[i]):sum(2)
    --jll[i]:add(b)
  end
  return jll
end

function kaggle_perplexity(jll, Y)
    local matrix = torch.zeros(Y:size(1), Y:size(2))
    for i=1, Y:size(1) do
        for j=1, Y:size(2) do
            idx = Y[i][j]
            matrix[i][j] = (jll[i][idx] * -1)
        end
    end
    return matrix:cdiv(torch.expand((torch.sum(matrix, 2)), Y:size(1), matrix:size(2)))
end

function valid_perplexity(jll, Y_correct, Y)
    perp = 0
    for i=1, Y_correct:size(1) do
        idx = Y[i][Y_correct[i]]
        perp = (torch.log(jll[i][idx])) + perp
    end
    print(perp)
end

function write2file(scores, fname)
    f = io.open(fname, "w")
    f:write("ID,Class1,CLass2,Class3,Class4,Class5,Class6,Class7,Class8,Class9,Class10,Class11,Class12,CLass13,Class14,Class15,Class16,Class17,Class18,Class19,Class20,Class21,Class22,Class23,CLass24,Class25,Class26,Class27,Class28,Class29,Class30,Class31,Class32,Class33,Class34,CLass35,Class36,Class37,Class38,Class39,Class40,Class41,Class42,Class43,Class44,Class45,CLass46,Class47,Class48,Class49,Class50\n")
    for i=1,scores:size(1) do
        s = tostring(i)
        for j=1, scores:size(2) do
            s = s .. "," .. tostring(scores[i][j])
        end
        f:write(s .. "\n")
    end
    f:close()
end

In [4]:
X_train = changeIndex(X_train, 5, nfeatures)
X_valid = changeIndex(X_valid, 5, nfeatures)
X_test = changeIndex(X_test, 5, nfeatures)

In [5]:
W = lookup(X_train, Y_train, window_size, 10001, nfeatures)

In [6]:
W_, b = CBM(W, Y_train, 1, 10001)

In [22]:
jll = predict(X_valid, W_, b)
jll:cdiv(torch.expand((torch.sum(jll, 2)), jll:size(1), jll:size(2)))

In [17]:
scores = perplexity(jll, q_valid)

In [119]:
write2file(scores, "pred_2.csv")

In [26]:
valid_perplexity(jll, kaggle_valid, q_valid)

-31709.444120994	



In [23]:
jll:sub(1,1)

Columns 1 to 10
0.0001 *
  0.8040  0.6695  1.0285  0.8218  0.7281  0.8761  0.8752  0.8332  0.8640  0.8659

Columns 11 to 20
0.0001 *
  0.8731  0.8632  0.8431  0.7704  0.9078  0.8620  0.8967  0.8455  0.8830  0.8597

Columns 21 to 30
0.0001 *
  0.8771  0.8512  0.8689  0.8679  0.8435  0.9203  0.9674  0.9459  0.8521  0.8928

Columns 31 to 40
0.0001 *
  0.8833  0.9235  0.8804  0.9814  0.8931  0.9305  0.7522  0.8802  0.7341  0.9009

Columns 41 to 50
0.0001 *
  0.8895  0.8224  0.9135  0.9086  0.8615  0.8868  0.7922  0.9383  0.7614  0.8752

Columns 51 to 60
0.0001 *
  0.8594  0.9278  0.8594  0.8631  0.7641  0.8367  0.8775  0.8913  0.8992  0.9223

Columns 61 to 70
0.0001 *
  0.9793  0.7985  0.7824  0.9124  0.9058  0.9237  0.9362  0.9067  0.9174  0.9328

Columns 71 to 80
0.0001 *
  0.8334  0.8421  0.8329  0.8424  0.8856  0.8111  0.8126  0.8619  0.9241  0.9136

Columns 81 to 90
0.0001 *
  0.9491  0.9375  0.8323  0.9317  0.9304  0.8719  0.8476  0.8337  0.9332  0.8964

Columns 91 to 100
0.0001 *
  

0.9601  0.8813  0.8889  0.8593  0.9549  0.9298  0.9498  0.9940

Columns 381 to 390
0.0001 *
  0.8782  0.8947  0.8676  0.9216  0.8660  0.9482  0.9382  0.9629  1.0179  0.9599

Columns 391 to 400
0.0001 *
  0.8754  0.8927  0.8964  0.9155  0.8863  0.9451  0.9351  1.0040  0.9479  0.9107

Columns 401 to 410
0.0001 *
  0.9045  0.9831  0.8930  0.8896  0.8994  0.9870  0.9540  1.0001  0.9190  0.8906

Columns 411 to 420
0.0001 *
  0.8917  0.9318  0.9607  0.9966  0.9324  0.9093  0.8802  0.9501  0.9680  0.9529

Columns 421 to 430
0.0001 *
  0.9453  0.9128  0.9301  0.8887  0.9266  0.9914  0.9425  0.8744  0.9836  0.9220

Columns 431 to 440
0.0001 *
  0.8909  0.9887  0.9333  0.8920  0.9286  0.9891  0.8976  0.9085  0.9967  0.9139

Columns 441 to 450
0.0001 *
  0.8713  0.9647  0.9256  0.9835  0.8934  0.8558  0.9493  0.9484  0.8824  0.9067

Columns 451 to 460
0.0001 *
  0.9017  0.8858  0.9074  0.9545  0.9966  0.9203  0.9289  0.9123  0.8978  0.9585

Columns 461 to 470
0.0001 *
  0.9422  0.8973  0.9103  0.

9861  0.9370  0.9234  0.9720  0.9729

Columns 751 to 760
0.0001 *
  0.9597  0.9301  0.9388  0.9588  0.9520  0.9171  0.8929  0.9665  0.9443  0.8675

Columns 761 to 770
0.0001 *
  0.9419  0.9292  0.9090  0.9877  0.9508  0.9229  0.9234  0.8974  0.9825  0.9474

Columns 771 to 780
0.0001 *
  0.9255  0.9353  0.9511  0.9454  0.9879  0.9000  0.9390  0.9826  0.9617  0.9692

Columns 781 to 790
0.0001 *
  1.0034  0.9182  0.9649  1.0035  0.9519  0.9475  0.9129  0.9649  0.9247  0.9992

Columns 791 to 800
0.0001 *
  0.9508  0.9388  0.9530  0.9241  0.9276  0.9353  0.9242  0.8965  0.9856  0.9404

Columns 801 to 810
0.0001 *
  1.0165  0.9322  0.9210  0.9122  0.9507  0.9825  0.9430  0.9519  0.9358  0.9309

Columns 811 to 820
0.0001 *
  0.9217  1.0034  0.9771  0.9485  0.9746  0.9378  0.9446  0.9825  0.9747  0.9330

Columns 821 to 830
0.0001 *
  0.9172  1.0165  0.9664  1.0165  0.9821  0.9406  0.9489  1.0082  0.9237  0.9369

Columns 831 to 840
0.0001 *
  0.9532  0.9718  0.9519  1.0297  0.9781  0.9078  1.00

886  0.9647  0.9822  0.9349  1.0160

Columns 1121 to 1130
0.0001 *
  0.9899  0.9876  0.9856  0.9404  0.9743  0.9661  0.9376  0.9876  0.9426  0.9952

Columns 1131 to 1140
0.0001 *
  0.9767  0.9766  0.9052  0.9767  0.9450  0.9766  0.9366  0.9584  0.9426  0.9613

Columns 1141 to 1150
0.0001 *
  0.9635  0.9876  0.9515  0.9409  1.0031  0.9660  0.9311  0.9899  0.9451  0.9876

Columns 1151 to 1160
0.0001 *
  1.0161  0.9821  0.9148  1.0031  0.9397  0.9514  0.9327  0.9098  0.9458  1.0085

Columns 1161 to 1170
0.0001 *
  0.9792  0.9515  0.9503  1.0160  0.9744  0.9408  0.9486  0.9821  0.9237  0.9767

Columns 1171 to 1180
0.0001 *
  1.0292  0.9613  0.9899  0.9349  0.9052  0.9473  0.9592  0.9647  0.9492  0.9874

Columns 1181 to 1190
0.0001 *
  0.9315  0.9535  1.0162  0.9896  0.9420  0.9514  0.9334  0.9791  0.9635  1.0085

Columns 1191 to 1200
0.0001 *
  0.9792  0.9647  0.9470  1.0030  0.9779  0.9354  0.9294  0.9591  0.9557  1.0161

Columns 1201 to 1210
0.0001 *
  0.9690  0.9425  0.9424  0.9592  0.9

 *
  1.0292  0.9556  0.9555  0.9556  0.9235  0.9610  0.9765  0.9645  0.9818  0.9819

Columns 1491 to 1500
0.0001 *
  0.9645  1.0083  1.0159  1.0160  0.9765  1.0028  0.9765  1.0028  0.9776  0.9985

Columns 1501 to 1510
0.0001 *
  0.9819  0.9424  0.9720  0.9266  0.9160  0.9722  0.9951  0.9722  1.0027  0.9948

Columns 1511 to 1520
0.0001 *
  0.9533  0.9591  0.9500  0.9986  1.0081  0.9896  0.9986  1.0160  0.9645  0.9401

Columns 1521 to 1530
0.0001 *
  0.9819  0.9514  0.9262  0.9896  1.0083  0.9060  0.9819  1.0160  0.9818  0.9896

Columns 1531 to 1540
0.0001 *
  0.9526  0.9401  1.0159  0.9513  0.9533  0.9951  1.0292  1.0028  0.9895  1.0160

Columns 1541 to 1550
0.0001 *
  0.9764  0.9495  0.9777  0.9401  1.0028  0.9686  0.9347  0.9776  0.9896  0.9818

Columns 1551 to 1560
0.0001 *
  0.9610  0.9366  0.9765  0.9895  0.9615  0.9951  1.0028  0.9686  0.9951  0.9391

Columns 1561 to 1570
0.0001 *
  0.9722  0.9544  0.9742  0.9458  0.9303  1.0292  0.9722  0.9712  0.9416  1.0291

Columns 1571 to 158

...

1 to 8440
0.0001 *
  1.0286  1.0286  1.0286  1.0154  1.0022  0.9945  1.0154  1.0154  1.0154  0.9980

Columns 8441 to 8450
0.0001 *
  1.0154  1.0154  0.9916  1.0154  1.0286  1.0286  1.0022  1.0154  1.0286  1.0286

Columns 8451 to 8460
0.0001 *
  1.0286  1.0286  0.9946  1.0154  1.0154  1.0077  1.0154  1.0286  1.0286  1.0286

Columns 8461 to 8470
0.0001 *
  0.9980  1.0286  0.9946  1.0286  1.0154  1.0286  1.0154  1.0154  1.0023  1.0286

Columns 8471 to 8480
0.0001 *
  1.0154  1.0286  1.0154  1.0154  1.0154  1.0154  1.0023  1.0154  1.0154  1.0286

Columns 8481 to 8490
0.0001 *
  1.0286  1.0023  0.9945  1.0154  1.0154  1.0286  1.0286  1.0286  1.0154  1.0286

Columns 8491 to 8500
0.0001 *
  0.9814  1.0286  1.0286  1.0286  1.0154  1.0154  1.0154  1.0154  1.0286  1.0154

Columns 8501 to 8510
0.0001 *
  1.0286  1.0286  0.9945  1.0154  1.0286  1.0286  0.9980  0.9945  1.0286  1.0154

Columns 8511 to 8520
0.0001 *
  1.0154  1.0154  1.0286  1.0286  1.0154  1.0286  1.0286  1.0286  1.0286  1.0154

Col

286  0.9848  1.0154  1.0286  1.0286

Columns 8801 to 8810
0.0001 *
  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  0.9891

Columns 8811 to 8820
0.0001 *
  1.0286  1.0286  1.0286  1.0286  0.9848  1.0154  1.0286  1.0286  1.0286  1.0286

Columns 8821 to 8830
0.0001 *
  1.0286  1.0154  1.0286  1.0286  1.0286  1.0286  1.0286  1.0154  1.0286  1.0154

Columns 8831 to 8840
0.0001 *
  1.0286  1.0022  1.0286  0.9945  1.0154  1.0022  1.0286  1.0286  0.9945  1.0286

Columns 8841 to 8850
0.0001 *
  1.0286  1.0286  1.0154  1.0022  1.0154  1.0286  1.0286  1.0154  1.0286  1.0286

Columns 8851 to 8860
0.0001 *
  1.0286  1.0286  1.0286  1.0154  1.0286  1.0286  1.0286  1.0154  1.0077  1.0022

Columns 8861 to 8870
0.0001 *
  1.0286  1.0286  1.0286  1.0154  1.0154  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 8871 to 8880
0.0001 *
  1.0286  1.0154  1.0286  1.0154  1.0286  1.0286  1.0154  1.0286  1.0154  1.0286

Columns 8881 to 8890
0.0001 *
  1.0154  1.0022  1.0286  0.9891  1.0

 *
  1.0154  1.0286  1.0154  1.0286  1.0286  1.0286  1.0286  1.0154  1.0286  1.0154

Columns 9171 to 9180
0.0001 *
  1.0286  1.0077  1.0286  1.0286  1.0286  1.0286  1.0286  0.9891  1.0286  1.0286

Columns 9181 to 9190
0.0001 *
  1.0286  1.0154  1.0286  1.0286  1.0286  1.0286  1.0154  1.0286  1.0286  1.0286

Columns 9191 to 9200
0.0001 *
  1.0286  1.0022  1.0286  1.0154  1.0286  1.0154  1.0154  1.0286  1.0286  1.0286

Columns 9201 to 9210
0.0001 *
  1.0286  1.0154  1.0154  1.0286  1.0286  1.0286  1.0286  1.0286  1.0154  1.0022

Columns 9211 to 9220
0.0001 *
  1.0154  1.0286  1.0286  1.0286  1.0286  1.0022  1.0286  1.0286  1.0154  0.9945

Columns 9221 to 9230
0.0001 *
  1.0286  1.0154  1.0154  1.0286  1.0286  1.0077  1.0286  1.0077  1.0154  1.0154

Columns 9231 to 9240
0.0001 *
  1.0286  1.0077  1.0286  1.0022  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 9241 to 9250
0.0001 *
  1.0286  1.0286  1.0154  1.0286  1.0286  1.0154  1.0154  1.0022  1.0154  1.0286

Columns 9251 to 926

286  0.9945  1.0154

Columns 9531 to 9540
0.0001 *
  1.0022  1.0154  1.0154  1.0286  1.0286  1.0286  1.0286  1.0286  1.0154  1.0286

Columns 9541 to 9550
0.0001 *
  1.0154  1.0154  1.0022  1.0286  1.0154  1.0154  1.0286  1.0286  1.0154  1.0022

Columns 9551 to 9560
0.0001 *
  0.9916  1.0286  1.0154  1.0077  1.0022  1.0286  1.0022  1.0286  1.0154  1.0286

Columns 9561 to 9570
0.0001 *
  1.0154  1.0286  1.0286  1.0154  1.0286  1.0286  1.0022  1.0022  1.0286  1.0286

Columns 9571 to 9580
0.0001 *
  1.0154  1.0154  1.0154  1.0286  1.0154  1.0286  1.0154  1.0286  1.0286  1.0286

Columns 9581 to 9590
0.0001 *
  1.0286  1.0286  1.0286  1.0286  1.0154  1.0286  1.0077  1.0286  1.0022  1.0286

Columns 9591 to 9600
0.0001 *
  1.0286  1.0286  1.0286  1.0154  1.0077  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 9601 to 9610
0.0001 *
  1.0154  1.0286  1.0286  1.0286  1.0154  1.0286  1.0286  1.0154  0.9980  1.0286

Columns 9611 to 9620
0.0001 *
  1.0286  1.0154  1.0286  1.0286  1.0077  0.9980  1.0

286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 9901 to 9910
0.0001 *
  1.0286  1.0154  1.0286  1.0286  1.0154  1.0286  1.0286  1.0154  1.0154  1.0286

Columns 9911 to 9920
0.0001 *
  1.0154  1.0286  1.0022  1.0286  1.0022  1.0154  1.0286  1.0286  1.0286  1.0286

Columns 9921 to 9930
0.0001 *
  1.0286  1.0154  1.0286  1.0286  1.0286  1.0286  1.0154  1.0154  1.0286  1.0022

Columns 9931 to 9940
0.0001 *
  1.0286  1.0286  1.0154  1.0286  1.0286  1.0286  1.0154  1.0286  1.0286  1.0286

Columns 9941 to 9950
0.0001 *
  1.0286  1.0286  1.0286  1.0286  1.0022  1.0286  1.0286  1.0286  0.9945  1.0286

Columns 9951 to 9960
0.0001 *
  1.0286  1.0286  1.0286  1.0286  1.0286  1.0022  1.0022  1.0286  1.0286  1.0286

Columns 9961 to 9970
0.0001 *
  1.0286  1.0154  1.0154  1.0285  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 9971 to 9980
0.0001 *
  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286  1.0286

Columns 9981 to 9990
0.0001 *
  1.0

In [10]:
kaggle_valid

 23
  6
 13
 16
 33
 23
 30
  3
 48
 49
 22
 27
 45
 47
 15
 11
 22
 12
 40
  1
  1
 33
 18
 47
 22
  6
  4
 44
 25
 30
 15
 28
 17
 50
 36
 30
 35
 19
 17
 21
  4
  9
 31
 46
 19
 32
 37
 39
 29
 28
 26
 36
 22
 23
 10
 45
 15
  5
 23
 27
 25
 45
 46
 31
  9
 40
  2
 10
 50
 49
 13
 31
 31
 34
 17
 21
  1
 50
 32
 24
  4
 32
 31
 45
 29
  8
  9
 28
 44
 48
 11
 18
 49
 21
 33
 47
 26
 29
 22
  1
 41
 34
  1
 40
 44
 19
  7
 35
 43
  5
 12
 28
 20
 37
 21
 33
 22
 29
 37
 14
 50
  1
 43
 26
 35
 17
 30
 10
 30
 15
  7
 10
 44
 25
 47
 13
 26
  4
 46
 35
  6
 41
  7
 33
 17
 14
 30
 30
  7
 34
 12
 34
 41
  1
 36
 25
 47
 37
  8
 17
  5
 39
 15
 44
 17
 44
 15
  7
 49
 41
 20
 40
 34
 29
  9
  3
 31
 50
 20
 14
 41
 12
 34
 20
 36
 45
 41
 10
  1
 10
 35
 16
 28
 39
 48
  7
 30
 28
 50
 42
  7
 14
 35
  3
 12
 37
 23
  2
 42
 19
 49
 44
  3
 42
  3
 27
  3
  7
 39
 42
 32
  3
 29
 22
 41
  9
 28
 21
 40
 39
 24
 15
 29
 11
 10
 28
  7
 38
 44
  8
 17
 40
 20
  4
 23
 12
 22
 23
 21
 28


 44
 43
 36
 34
 18
 30
 32
 29
 16
 44
 23
 39
  9
 11
 20
 37
 30
 22
 46
 14
 31
 18
 17
 36
  4
 34
 38
  2
 22
 43
 26
  3
 17
 21
 20
 23
 45
 41
 13
 36
 20
 22
 14
 31
 37
 37
 42
  7
 19
 15
  5
 45
 31
 21
 20
 23
 41
 12
 45
 36
 38
  3
  6
 42
 27
 23
 12
 42
 15
  4
 24
 28
 34
 22
 17
 48
 47
 12
 26
 47
 22
 13
 40
 22
 35
 30
  9
 46
 21
 36
 20
 38
  4
  6
 47
 44
 37
 46
 15
  7
 24
 29
  8
 44
 26
 50
 30
 30
 34
 47
  4
 46
 33
 24
 33
 40
 41
 46
 10
  3
  9
 31
 41
 27
 31
 19
 42
 48
  3
 46
 22
 30
 26
  3
  4
 32
 47
 47
 23
  5
 28
 18
 30
 20
  3
 35
 10
 30
  9
 44
 43
 29
 39
 16
 10
  9
 15
 22
  3
 48
 34
  1
 24
  4
 49
 49
 17
 26
 45
  1
 36
 16
 42
 45
 26
 46
  8
 10
 12
 25
 33
 21
 39
 39
 42
 40
 19
 18
 26
 43
 34
  8
 19
 37
  3
  6
  5
 42
  5
 50
 13
  8
 36
 20
 11
 14
 34
  9
  1
  4
 48
 49
 21
 49
 30
 24
 42
 23
  4
 27
 20
  4
 16
 38
 44
 47
 32
 49
  8
 19
 32
 23
 37
 43
  4
 26
 19
 42
 30
 23
 31
  4
 34
 29
 47
 33
 40
  1
 13
  6


  9
 15
 45
 42
  3
 26
 19
 10
  6
 21
 38
 26
 14
 37
  1
 24
  2
 25
 32
 37
 45
  5
 14
 30
 17
 30
 36
 45
 30
 21
 43
  8
 31
 29
 30
 29
 24
 46
  1
 39
 39
 32
 43
 16
  1
 15
 27
 49
 17
 33
  2
 47
  7
 12
  3
 25
 26
 17
 43
 43
  3
 10
 43
 49
 17
 26
 10
 50
 27
 45
 26
 49
 33
 35
 15
 16
 17
 32
 50
 45
 46
 26
 46
 41
 41
  9
  4
 50
 10
 42
 34
 29
 47
 18
 37
 44
 20
  1
 24
 23
 43
 15
  6
  7
 11
 36
 41
 42
 33
 46
  1
 10
  6
 43
  9
 45
  2
 28
 15
 13
 35
 17
 20
  8
 50
 28
 23
 23
 43
 22
 36
 20
 41
  1
  9
 17
 16
 47
 47
 49
 14
 32
 46
 36
  2
 10
 34
 39
 46
 19
 26
  9
 23
  4
 37
 44
 22
 22
 17
 46
 44
 41
 48
 14
 37
 44
 36
  1
 12
  4
 34
 18
 28
 32
 19
 38
 11
 19
 23
 36
 14
 10
 40
 47
 39
 10
 24
  4
 47
 29
 19
  1
  4
 10
 14
 41
 41
 21
 40
 41
 20
 50
 35
 26
  5
 33
 33
  3
  1
 48
 35
 31
 24
 28
 48
 24
 16
 20
 24
 38
 17
 24
 24
 27
 23
 33
 39
 15
 28
 20
 24
 14
  2
 46
 12
 20
 19
 31
 24
 17
 25
 50
 12
 37
 21
 16
 22
 39
 11
 15


 41
  1
 39
  8
 33
 38
 32
 50
 40
 22
 45
 10
 33
  2
 50
 46
 10
 18
  7
 33
 49
 36
 43
 14
 39
 24
 50
 42
 29
  4
 10
 15
 28
 31
 30
  9
 26
 38
 31
 32
 13
 47
 48
 17
 12
  7
 41
  2
  4
  8
  9
 35
 49
  6
  3
 49
 11
 23
 29
 38
 25
 18
 29
 40
 11
 12
 30
  4
  5
 40
 17
 46
 45
  6
 26
 10
 14
 25
 40
 22
 43
 25
  4
  3
 21
 49
 50
 36
 12
 45
 15
 43
 10
 29
 20
 41
 16
 27
 50
 41
 31
 43
 32
  5
 13
 13
 13
 46
 20
 39
 13
 26
 37
 22
  2
  1
 45
  4
 24
  2
 20
 15
 44
  5
 17
 11
 10
 30
 41
 26
 17
 27
 15
 42
 34
 37
 20
 23
 50
 20
 48
  5
  4
 23
 41
 12
 35
 47
 33
 27
  7
 17
 38
 11
 30
 15
 13
 10
 40
 10
 17
  4
 40
 30
 10
  8
 20
 18
 22
 31
 24
 45
 25
 21
 45
 49
 46
 37
 19
 13
 48
 20
 24
 17
 30
 13
 36
 33
 34
 10
 38
  1
 15
 24
  3
 19
 38
 14
 23
 35
 26
 18
 12
 29
 20
 38
 14
 23
 24
 26
 42
 13
 50
 21
  3
  8
  5
  8
 17
 11
 13
  3
  3
 43
 45
 24
  9
 29
 50
  7
  1
 13
 24
 20
  6
 29
 18
  9
 43
  7
 23
  1
 30
  8
  2
 35
 45
 36
 36
 31
